In [ ]:
!git clone https://github.com/boberle/corefconversion

In [ ]:
num = "55" #enter the number of the document

Change the extension to .sarc

In [137]:
!cp /Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}.txt /Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}_ishaan.sarc

In [138]:
!cp /Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}.txt /Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}_jonathan.sarc

Convert from .sarc to BRAT

In [139]:
#convert from sarc to brat format
!python3 ./corefconversion/sacr2ann.py /Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}_ishaan.sarc --txt /Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}_ishaan.txt --ann /Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}_ishaan.ann

In [140]:
#convert from sarc to brat format
!python3 ./corefconversion/sacr2ann.py /Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}_jonathan.sarc --txt /Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}_jonathan.txt --ann /Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}_jonathan.ann

#note the cluster_a is predicted and cluster_b is ground truth cluster

In [142]:
import re
from collections import defaultdict
from scipy.optimize import linear_sum_assignment
import numpy as np
from itertools import combinations

In [143]:
def parse_sarc_file(filename):
    mentions = {}
    coreferences = []
    
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("T"):  # Mention
                parts = line.strip().split('\t')
                mention_id = parts[0]
                mention_text = parts[2]
                # Capture position as a unique identifier for each mention
                mention_pos = tuple(map(int, parts[1].split()[1:3]))
                mentions[mention_id] = (mention_text, mention_pos)
                
            elif line.startswith("R"):  # Coreference link
                match = re.match(r"R\d+\s+Coreference\s+Arg1:(T\d+)\s+Arg2:(T\d+)", line)
                if match:
                    arg1, arg2 = match.groups()
                    coreferences.append((arg1, arg2))
    
    return mentions, coreferences

def build_clusters(mentions, coref_links):
    clusters = defaultdict(list)
    
    mention_to_cluster = {}
    cluster_id = 0
    
    for arg1, arg2 in coref_links:
        if arg1 not in mention_to_cluster and arg2 not in mention_to_cluster:
            # New cluster
            mention_to_cluster[arg1] = cluster_id
            mention_to_cluster[arg2] = cluster_id
            clusters[cluster_id].extend([mentions[arg1], mentions[arg2]])
            cluster_id += 1
        elif arg1 in mention_to_cluster:
            cluster_idx = mention_to_cluster[arg1]
            clusters[cluster_idx].append(mentions[arg2])
            mention_to_cluster[arg2] = cluster_idx
        elif arg2 in mention_to_cluster:
            cluster_idx = mention_to_cluster[arg2]
            clusters[cluster_idx].append(mentions[arg1])
            mention_to_cluster[arg1] = cluster_idx
            
    return list(clusters.values())

# Convert clusters to sets of (name, position) tuples for comparison
def clusters_to_sets(clusters):
    return [set(cluster) for cluster in clusters]

def calculate_precision_recall(predicted_clusters, ground_truth_clusters):
    predicted_sets = clusters_to_sets(predicted_clusters)
    ground_truth_sets = clusters_to_sets(ground_truth_clusters)
    
    correct = sum(1 for pc in predicted_sets if pc in ground_truth_sets)
    precision = correct / len(predicted_sets) if predicted_sets else 0
    recall = correct / len(ground_truth_sets) if ground_truth_sets else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1

# Process the files and evaluate
# mentions_b, coref_links_b = parse_sarc_file('/Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/90_ishaan.ann')
mentions_a, coref_links_a = parse_sarc_file(f'/Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}_ishaan.ann')
mentions_b, coref_links_b = parse_sarc_file(f'/Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}_jonathan.ann')


clusters_a = build_clusters(mentions_a, coref_links_a)
clusters_b = build_clusters(mentions_b, coref_links_b)

precision_a, recall_a, f1_a = calculate_precision_recall(clusters_a, clusters_b)



In [144]:
def muc_score(clusters_a, clusters_b):
    """
    Calculate MUC score for coreference resolution.
    
    Args:
        clusters_a, clusters_b: Lists of clusters, where each cluster contains tuples of (mention_text, position)
    """
    def get_mention_pairs(cluster):
        """Convert a cluster into a set of mention pairs based on positions"""
        mentions = sorted(cluster, key=lambda x: x[1])  # Sort by position
        return set(combinations(mentions, 2))
    
    # Convert clusters to sets and get all mention pairs
    clusters_a = [set(cluster) for cluster in clusters_a]
    clusters_b = [set(cluster) for cluster in clusters_b]
    
    # Create mention to cluster mappings
    mention_to_cluster_a = {}
    mention_to_cluster_b = {}
    
    for i, cluster in enumerate(clusters_a):
        for mention in cluster:
            mention_to_cluster_a[mention] = i
            
    for i, cluster in enumerate(clusters_b):
        for mention in cluster:
            mention_to_cluster_b[mention] = i
    
    # Calculate links for each cluster
    links_a = set()
    links_b = set()
    
    for cluster in clusters_a:
        links_a.update(get_mention_pairs(cluster))
        
    for cluster in clusters_b:
        links_b.update(get_mention_pairs(cluster))
    
    # Calculate correct links (intersection)
    correct_links = links_a.intersection(links_b)
    
    # Calculate MUC scores
    precision = len(correct_links) / len(links_a) if links_a else 0
    recall = len(correct_links) / len(links_b) if links_b else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

def b3_score(predicted_clusters, ground_truth_clusters):
    """
    Calculate B³ score for coreference resolution.
    
    Args:
        predicted_clusters: List of clusters, where each cluster contains tuples of (mention_text, position)
        ground_truth_clusters: List of clusters, where each cluster contains tuples of (mention_text, position)
    """
    def get_mention_to_cluster_mapping(clusters):
        mapping = {}
        # Convert each cluster to a set if it isn't already
        clusters = [set(cluster) for cluster in clusters]
        for cluster in clusters:
            for mention in cluster:
                mapping[mention] = cluster
        return mapping

    # Create mappings
    pred_mapping = get_mention_to_cluster_mapping(predicted_clusters)
    gt_mapping = get_mention_to_cluster_mapping(ground_truth_clusters)
    
    total_precision = 0
    total_recall = 0
    
    # Use the union of all mentions to ensure we don't miss any
    all_mentions = set(pred_mapping.keys()).union(set(gt_mapping.keys()))
    num_mentions = len(all_mentions)
    
    for mention in all_mentions:
        if mention in pred_mapping and mention in gt_mapping:
            predicted_cluster = pred_mapping[mention]
            ground_truth_cluster = gt_mapping[mention]
            
            intersection_size = len(predicted_cluster.intersection(ground_truth_cluster))
            precision_for_mention = intersection_size / len(predicted_cluster)
            recall_for_mention = intersection_size / len(ground_truth_cluster)
            
            total_precision += precision_for_mention
            total_recall += recall_for_mention
        elif mention in pred_mapping:
            # Mention in predicted but not in ground truth
            total_precision += 0
        elif mention in gt_mapping:
            # Mention in ground truth but not in predicted
            total_recall += 0

    precision = total_precision / num_mentions if num_mentions > 0 else 0
    recall = total_recall / num_mentions if num_mentions > 0 else 0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Example usage:
def evaluate_coreference(predicted_clusters, ground_truth_clusters):
    """
    Evaluate using both MUC and B³ metrics
    """
    # B³ scores
    b3_p, b3_r, b3_f1 = b3_score(predicted_clusters, ground_truth_clusters)
    print("\nB³ Scores:")
    print(f"Precision: {b3_p:.3f}")
    print(f"Recall: {b3_r:.3f}")
    print(f"F1: {b3_f1:.3f}")
    
    # MUC scores
    muc_p, muc_r, muc_f1 = muc_score(predicted_clusters, ground_truth_clusters)
    print("\nMUC Scores:")
    print(f"Precision: {muc_p:.3f}")
    print(f"Recall: {muc_r:.3f}")
    print(f"F1: {muc_f1:.3f}")

     # Basic precision/recall
    precision, recall, f1 = calculate_precision_recall(clusters_a, clusters_b)
    print("\nBasic Evaluation:")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1: {f1:.3f}")
    

# Usage
evaluate_coreference(clusters_a, clusters_b)

In [145]:
def long_distance_evaluation(annotations_file_a, annotations_file_b, text_doc):
    # Parse both annotation files
    mentions_a, coref_links_a = parse_sarc_file(annotations_file_a)
    mentions_b, coref_links_b = parse_sarc_file(annotations_file_b)

    # Parse paragraphs
    with open(text_doc, 'r') as file:
        text = file.read()
    paragraphs = text.split('\n\n')
    
    def get_paragraph_index(position):
        char_count = 0
        for idx, para in enumerate(paragraphs):
            char_count += len(para) + 2  # +2 for '\n\n'
            if position <= char_count:
                return idx
        return len(paragraphs) - 1
    
    # Filter long-distance links for both sets
    def filter_long_distance(coref_links, mentions):
        long_distance_links = []
        for arg1, arg2 in coref_links:
            pos1 = mentions[arg1][1][0]  # Get start position of first mention
            pos2 = mentions[arg2][1][0]  # Get start position of second mention
            
            para1 = get_paragraph_index(pos1)
            para2 = get_paragraph_index(pos2)
            
            if para1 != para2:
                long_distance_links.append((arg1, arg2))
                
        return long_distance_links

    coref_links_a_long = filter_long_distance(coref_links_a, mentions_a)
    coref_links_b_long = filter_long_distance(coref_links_b, mentions_b)
    
    # Build clusters using filtered long-distance coreferences
    clusters_a = build_clusters(mentions_a, coref_links_a_long)
    clusters_b = build_clusters(mentions_b, coref_links_b_long)

    print(clusters_a)
    print(clusters_b)

    def get_long_distance_pairs(cluster):
        """Convert a cluster into a set of long-distance mention pairs"""
        mentions = sorted(cluster, key=lambda x: x[1])
        # pairs = set()
        # for m1, m2 in combinations(mentions, 2):
        #     pos1 = m1[1][0]
        #     pos2 = m2[1][0]
        #     para1 = get_paragraph_index(pos1)
        #     para2 = get_paragraph_index(pos2)
        #     if para1 != para2:
        #         pairs.add((m1, m2))
        # return pairs
        return set(combinations(mentions, 2))

    def long_distance_muc(clusters_a, clusters_b):
        # Convert clusters to sets
        clusters_a = [set(cluster) for cluster in clusters_a]
        clusters_b = [set(cluster) for cluster in clusters_b]
        
        # Get all long-distance mention pairs
        links_a = set()
        links_b = set()
        
        for cluster in clusters_a:
            links_a.update(get_long_distance_pairs(cluster))
            
        for cluster in clusters_b:
            links_b.update(get_long_distance_pairs(cluster))
        
        # Calculate correct links (intersection)
        correct_links = links_a.intersection(links_b)
        
        # Calculate MUC scores
        precision = len(correct_links) / len(links_a) if links_a else 0
        recall = len(correct_links) / len(links_b) if links_b else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return precision, recall, f1

    # Calculate both regular and MUC scores
    muc_p, muc_r, muc_f1 = long_distance_muc(clusters_a, clusters_b)
    
    return {
        'muc_scores': (muc_p, muc_r, muc_f1)
    }

In [146]:
txt_path = f"/Users/ishaan/Documents/ruihong_lab/legal_project/dataset/contacts_events_data/data/contract_events/{num}.txt"
ann_a = f'/Users/ishaan/Documents/ruihong_lab/legal_project/ishaan_annotations/{num}_ishaan.ann'
ann_b = f'/Users/ishaan/Documents/ruihong_lab/legal_project/jonathan_annotations/only_long_dist/{num}_jonathan.ann'
long_distance_evaluation(ann_a, ann_b, txt_path)

In [147]:
mentions_a , corefs_a =  parse_sarc_file(ann_a)

In [148]:
def get_long_only_clusters(annotations_file_a, annotations_file_b, text_doc):
    # Parse both annotation files
    mentions_a, coref_links_a = parse_sarc_file(annotations_file_a)
    mentions_b, coref_links_b = parse_sarc_file(annotations_file_b)

    # Parse paragraphs
    with open(text_doc, 'r') as file:
        text = file.read()
    paragraphs = text.split('\n\n')
    
    def get_paragraph_index(position):
        char_count = 0
        for idx, para in enumerate(paragraphs):
            char_count += len(para) + 2  # +2 for '\n\n'
            if position <= char_count:
                return idx
        return len(paragraphs) - 1
    
    # Filter long-distance links for both sets
    def filter_long_distance(coref_links, mentions):
        long_distance_links = []
        for arg1, arg2 in coref_links:
            pos1 = mentions[arg1][1][0]  # Get start position of first mention
            pos2 = mentions[arg2][1][0]  # Get start position of second mention
            
            para1 = get_paragraph_index(pos1)
            para2 = get_paragraph_index(pos2)
            
            if para1 != para2:
                long_distance_links.append((arg1, arg2))
                
        return long_distance_links

    coref_links_a_long = filter_long_distance(coref_links_a, mentions_a)
    coref_links_b_long = filter_long_distance(coref_links_b, mentions_b)
    
    # Build clusters using filtered long-distance coreferences
    clusters_a = build_clusters(mentions_a, coref_links_a_long)
    clusters_b = build_clusters(mentions_b, coref_links_b_long)

    return clusters_a, clusters_b

In [149]:
long_cluster_a, long_cluster_b = get_long_only_clusters(ann_a, ann_b, txt_path)

In [150]:
b3_score(long_cluster_a, long_cluster_b)

In [151]:
long_cluster_a

In [152]:
long_cluster_b